In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier

import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.hashes import SHA256

In [2]:
#membaca data berformat CSV yang sudah disimpan didalam folder google drive
data = pd.read_csv('Churn_Modelling.csv')
data.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [3]:
# Drop unneeded columns
#data.drop(["RowNumber",'CustomerId','Surname'], axis=1, inplace=True)
data['EstimatedSalary'] = data['EstimatedSalary'].astype(int)
data['Balance'] = data['Balance'].astype(int)

In [4]:
data.dtypes

RowNumber           int64
CustomerId          int64
Surname            object
CreditScore         int64
Geography          object
Gender             object
Age                 int64
Tenure              int64
Balance             int32
NumOfProducts       int64
HasCrCard           int64
IsActiveMember      int64
EstimatedSalary     int32
Exited              int64
dtype: object

In [6]:
X = data.drop(['Exited'], axis =1)
y = data['Exited']

In [7]:
# Function to generate a secure key using PBKDF2
def generate_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=SHA256(),
        length=32,
        salt=salt,
        iterations=100000,  # You can adjust the number of iterations based on your security requirements
        backend=default_backend()
    )
    print(password)
    return kdf.derive(password)

# Encrypt using AES algorithm in CFB mode
def encrypt_df(df, key, iv):
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=backend)

    encrypted_data = {}
    for col in df.columns:
        encrypted_col = []
        for value in df[col]:
            encryptor = cipher.encryptor()
            # Convert the value to bytes
            value_bytes = str(value).encode('utf-8')
            # Encrypt the data
            encrypted_value = encryptor.update(value_bytes) + encryptor.finalize()
            encrypted_col.append(base64.b64encode(encrypted_value).decode())

        encrypted_data[col] = encrypted_col

    encrypted_df = pd.DataFrame(encrypted_data)
    return encrypted_df


# Convert base64-encoded DataFrame to integer
def base64_to_int(df):
    return df.applymap(lambda x: int.from_bytes(base64.b64decode(x.encode()), byteorder='big'))

# Example password and salt (for demonstration purposes only)
password = b'SecurePassword123'
salt =b'D\x80\xe3\x94N\n\xffA\x05K\xee\xb0\xf7M\xdd\xdf'

print(salt)
# Generate a secure key
key = generate_key(password, salt)

# Generate a random IV (Initialization Vector)
iv =b'\x01\xebR@G\xd6\xdd@\xff\xcc\xb3:\xd6\xad\xae\xbf'


print(iv)
# Encrypt the DataFrame
data_encrypted = encrypt_df(X, key, iv)

# Convert DataFrame to base64
data_base64 = data_encrypted.applymap(lambda x: base64.b64encode(x.encode()).decode())

# Convert base64-encoded DataFrame to integer
data_final = base64_to_int(data_base64) 

b'D\x80\xe3\x94N\n\xffA\x05K\xee\xb0\xf7M\xdd\xdf'
b'SecurePassword123'
b'\x01\xebR@G\xd6\xdd@\xff\xcc\xb3:\xd6\xad\xae\xbf'


In [16]:
b'D\x80\xe3\x94N\n\xffA\x05K\xee\xb0\xf7M\xdd\xdf'

b'D\x80\xe3\x94N\n\xffA\x05K\xee\xb0\xf7M\xdd\xdf'

In [8]:
X=data_final

In [9]:
result = pd.concat([X, y], axis=1)

In [49]:
rs=result.drop(columns=['RowNumber','Exited'])

In [50]:
rs

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,34757157812124201130195894589,15891586980470449293777068861,1866020713,3473737896946584693,3473525673970054005,1868836925,1886862653,1884372285,1883323709,1883323709,1883323709,8092457744812038010
1,34757157812200773297501588797,3700095442385059133,1866018666,8678483994702985277,3473525673970054005,1868837949,1883323709,8242847815683889469,1883323709,1884372285,1883323709,8092466536624052021
2,34757157812112117475931806013,3634202011178712381,1867394928,3473737896946584693,3473525673970054005,1868836925,1917926717,8092531377746699575,1885814077,1883323709,1884372285,8092466532345858870
3,34757157793681938644388501821,3545532686216412477,1866031721,3473737896946584693,3473525673970054005,1886673213,1883323709,1884372285,1886862653,1884372285,1884372285,8236655366193767229
4,34757157793677468330809061181,15562962329902754878689915965,1919185522,8678483994702985277,3473525673970054005,1868855101,1886862653,8092462160052372279,1883323709,1883323709,1883323709,8031152279311701309
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,34757157812128720105803772733,15608679615301566458512368445,1869900401,3473737896946584693,3623486969687260477,1886673213,1866546493,1884372285,1886862653,1883323709,1884372285,8236642197842249021
9996,34757157830636830418870419773,15846918300514344113835505988,1867396980,3473737896946584693,3623486969687260477,1886678845,1884174397,8020465013385553725,1883323709,1883323709,1883323709,8092457744827378041
9997,34757157830719563931480572221,844718453,1869885289,3473737896946584693,3473525673970054005,1886677821,1869036861,1884372285,1883323709,1884372285,1883323709,8026597002637824317
9998,34757157812272811100313445181,37272647937738898482590415184,1869900400,14990006938789327562078240061,3623486969687260477,1868836925,1885814077,8031169871479918909,1886862653,1883323709,1884372285,8236577300904241981


In [14]:
from sqlalchemy import create_engine

In [38]:
from urllib.parse import quote
from sqlalchemy import create_engine

# Replace 'username', 'password', 'database', 'hostname', and 'port' with your MySQL credentials
username = 'root'
password = 'NithiArr@1920'
database = 'project'
hostname = 'localhost'
port = '3306'

# URL-encode only special characters in the password
url_encoded_password = ''.join(quote(char, safe='') if char not in ('_', '.') else char for char in password)

# Construct the connection string
engine = create_engine(f"mysql+mysqlconnector://{username}:{url_encoded_password}@{hostname}:{port}/{database}")


In [40]:
engine.connect()

In [51]:
table_name = 'project.base_customer'  # Replace with your desired table name

rs.to_sql(name=table_name, con=engine, if_exists='replace', index=False)


DataError: (mysql.connector.errors.DataError) 1264 (22003): Out of range value for column 'CustomerId' at row 1
[SQL: INSERT INTO `project.base_customer` (`CustomerId`, `Surname`, `CreditScore`, `Geography`, `Gender`, `Age`, `Tenure`, `Balance`, `NumOfProducts`, `HasCrCard`, `IsActiveMember`, `EstimatedSalary`) VALUES (%(CustomerId)s, %(Surname)s, %(CreditScore)s, %(Geography)s, %(Gender)s, %(Age)s, %(Tenure)s, %(Balance)s, %(NumOfProducts)s, %(HasCrCard)s, %(IsActiveMember)s, %(EstimatedSalary)s)]
[parameters: ({'CustomerId': 34757157812124201130195894589, 'Surname': 15891586980470449293777068861, 'CreditScore': 1866020713, 'Geography': 3473737896946584693, 'Gender': 3473525673970054005, 'Age': 1868836925, 'Tenure': 1886862653, 'Balance': 1884372285, 'NumOfProducts': 1883323709, 'HasCrCard': 1883323709, 'IsActiveMember': 1883323709, 'EstimatedSalary': 8092457744812038010}, {'CustomerId': 34757157812200773297501588797, 'Surname': 3700095442385059133, 'CreditScore': 1866018666, 'Geograp

In [65]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Replace 'username', 'password', 'database', 'hostname', and 'port' with your MySQL credentials
username = 'root'
password = 'nithi123'
database = 'proj'
hostname = 'localhost'
port = 3306  # Note: Port should be an integer


# Connect to MySQL
connection = mysql.connector.connect(
    host=hostname,
    user=username,
    password=password,
    database=database,
    port=port
)

# Create a MySQL cursor
cursor = connection.cursor()

# Drop the table if it exists


# Use pandas to insert data into MySQL
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{hostname}:{port}/{database}")
rs.to_sql(name='base_customer', con=engine, index=False, if_exists='append')

# Close the MySQL connection
connection.close()


In [59]:
rs.dtypes

CustomerId         object
Surname            object
CreditScore         int64
Geography          object
Gender              int64
Age                 int64
Tenure              int64
Balance             int64
NumOfProducts       int64
HasCrCard           int64
IsActiveMember      int64
EstimatedSalary     int64
dtype: object

In [27]:
import pandas as pd
import csv

# Assuming 'df' is your DataFrame
result.to_csv('output.csv', index=False, quoting=csv.QUOTE_NONE, encoding='utf-8')


In [25]:
import pandas as pd

# Assuming 'result' is the concatenated DataFrame
result['CustomerId'] = result['CustomerId'].astype(str)

# Save to CSV without scientific notation
result.to_csv('output3.csv', index=False)


In [8]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1866020713,3473737896946584693,3473525673970054005,1868836925,1886862653,1884372285,1883323709,1883323709,1883323709,8092457744812038010
1,1866018666,8678483994702985277,3473525673970054005,1868837949,1883323709,8242847815683889469,1883323709,1884372285,1883323709,8092466536624052021
2,1867394928,3473737896946584693,3473525673970054005,1868836925,1917926717,8092531377746699575,1885814077,1883323709,1884372285,8092466532345858870
3,1866031721,3473737896946584693,3473525673970054005,1886673213,1883323709,1884372285,1886862653,1884372285,1884372285,8236655366193767229
4,1919185522,8678483994702985277,3473525673970054005,1868855101,1886862653,8092462160052372279,1883323709,1883323709,1883323709,8031152279311701309
...,...,...,...,...,...,...,...,...,...,...
9995,1869900401,3473737896946584693,3623486969687260477,1886673213,1866546493,1884372285,1886862653,1883323709,1884372285,8236642197842249021
9996,1867396980,3473737896946584693,3623486969687260477,1886678845,1884174397,8020465013385553725,1883323709,1883323709,1883323709,8092457744827378041
9997,1869885289,3473737896946584693,3473525673970054005,1886677821,1869036861,1884372285,1883323709,1884372285,1883323709,8026597002637824317
9998,1869900400,14990006938789327562078240061,3623486969687260477,1868836925,1885814077,8031169871479918909,1886862653,1883323709,1884372285,8236577300904241981


In [9]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [10]:
# Split the Data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(x_train.shape)
print(x_test.shape)

(8000, 10)
(2000, 10)


In [11]:
# CatBoostClassifier
cat_model = CatBoostClassifier()

# Fit the model
cat_model.fit(x_train, y_train)

# Predictions
y_pred_cat = cat_model.predict(x_test)

# Print classification report
print(classification_report(y_test, y_pred_cat))


Learning rate set to 0.025035
0:	learn: 0.6729430	total: 144ms	remaining: 2m 23s
1:	learn: 0.6558148	total: 149ms	remaining: 1m 14s
2:	learn: 0.6382620	total: 154ms	remaining: 51.1s
3:	learn: 0.6214073	total: 159ms	remaining: 39.5s
4:	learn: 0.6072964	total: 164ms	remaining: 32.7s
5:	learn: 0.5909382	total: 170ms	remaining: 28.1s
6:	learn: 0.5763669	total: 174ms	remaining: 24.7s
7:	learn: 0.5633927	total: 181ms	remaining: 22.5s
8:	learn: 0.5506358	total: 188ms	remaining: 20.7s
9:	learn: 0.5391333	total: 193ms	remaining: 19.2s
10:	learn: 0.5280745	total: 199ms	remaining: 17.9s
11:	learn: 0.5186937	total: 205ms	remaining: 16.8s
12:	learn: 0.5084214	total: 211ms	remaining: 16s
13:	learn: 0.5000883	total: 216ms	remaining: 15.2s
14:	learn: 0.4912795	total: 221ms	remaining: 14.5s
15:	learn: 0.4841664	total: 227ms	remaining: 14s
16:	learn: 0.4779682	total: 231ms	remaining: 13.3s
17:	learn: 0.4711461	total: 236ms	remaining: 12.9s
18:	learn: 0.4647274	total: 242ms	remaining: 12.5s
19:	learn: 0.

173:	learn: 0.3249529	total: 1.16s	remaining: 5.5s
174:	learn: 0.3248166	total: 1.16s	remaining: 5.49s
175:	learn: 0.3245649	total: 1.17s	remaining: 5.48s
176:	learn: 0.3244946	total: 1.18s	remaining: 5.47s
177:	learn: 0.3243524	total: 1.18s	remaining: 5.46s
178:	learn: 0.3242390	total: 1.19s	remaining: 5.45s
179:	learn: 0.3241385	total: 1.19s	remaining: 5.44s
180:	learn: 0.3239576	total: 1.2s	remaining: 5.43s
181:	learn: 0.3238180	total: 1.2s	remaining: 5.42s
182:	learn: 0.3236320	total: 1.21s	remaining: 5.4s
183:	learn: 0.3234801	total: 1.22s	remaining: 5.39s
184:	learn: 0.3234187	total: 1.22s	remaining: 5.38s
185:	learn: 0.3232047	total: 1.23s	remaining: 5.36s
186:	learn: 0.3230414	total: 1.23s	remaining: 5.35s
187:	learn: 0.3229526	total: 1.24s	remaining: 5.34s
188:	learn: 0.3228403	total: 1.24s	remaining: 5.33s
189:	learn: 0.3227087	total: 1.25s	remaining: 5.32s
190:	learn: 0.3226151	total: 1.25s	remaining: 5.3s
191:	learn: 0.3223824	total: 1.26s	remaining: 5.29s
192:	learn: 0.322

347:	learn: 0.3031371	total: 2.15s	remaining: 4.04s
348:	learn: 0.3030149	total: 2.16s	remaining: 4.03s
349:	learn: 0.3028620	total: 2.17s	remaining: 4.03s
350:	learn: 0.3027797	total: 2.17s	remaining: 4.02s
351:	learn: 0.3026777	total: 2.18s	remaining: 4.01s
352:	learn: 0.3025215	total: 2.19s	remaining: 4.01s
353:	learn: 0.3024124	total: 2.19s	remaining: 4s
354:	learn: 0.3022366	total: 2.2s	remaining: 4s
355:	learn: 0.3020995	total: 2.2s	remaining: 3.99s
356:	learn: 0.3019772	total: 2.21s	remaining: 3.98s
357:	learn: 0.3018279	total: 2.22s	remaining: 3.98s
358:	learn: 0.3017249	total: 2.22s	remaining: 3.97s
359:	learn: 0.3015900	total: 2.23s	remaining: 3.96s
360:	learn: 0.3013848	total: 2.23s	remaining: 3.96s
361:	learn: 0.3012387	total: 2.25s	remaining: 3.96s
362:	learn: 0.3011564	total: 2.25s	remaining: 3.96s
363:	learn: 0.3010420	total: 2.26s	remaining: 3.95s
364:	learn: 0.3009393	total: 2.27s	remaining: 3.94s
365:	learn: 0.3008102	total: 2.27s	remaining: 3.94s
366:	learn: 0.300724

516:	learn: 0.2834251	total: 3.55s	remaining: 3.31s
517:	learn: 0.2833312	total: 3.56s	remaining: 3.31s
518:	learn: 0.2832177	total: 3.57s	remaining: 3.31s
519:	learn: 0.2831195	total: 3.58s	remaining: 3.3s
520:	learn: 0.2830450	total: 3.59s	remaining: 3.3s
521:	learn: 0.2828970	total: 3.6s	remaining: 3.3s
522:	learn: 0.2827798	total: 3.61s	remaining: 3.29s
523:	learn: 0.2826661	total: 3.62s	remaining: 3.29s
524:	learn: 0.2825469	total: 3.63s	remaining: 3.29s
525:	learn: 0.2824489	total: 3.64s	remaining: 3.28s
526:	learn: 0.2823507	total: 3.65s	remaining: 3.28s
527:	learn: 0.2822290	total: 3.66s	remaining: 3.27s
528:	learn: 0.2821340	total: 3.67s	remaining: 3.27s
529:	learn: 0.2820159	total: 3.68s	remaining: 3.27s
530:	learn: 0.2819348	total: 3.69s	remaining: 3.26s
531:	learn: 0.2818158	total: 3.7s	remaining: 3.26s
532:	learn: 0.2817153	total: 3.71s	remaining: 3.25s
533:	learn: 0.2816467	total: 3.73s	remaining: 3.25s
534:	learn: 0.2815305	total: 3.73s	remaining: 3.25s
535:	learn: 0.281

677:	learn: 0.2680557	total: 4.99s	remaining: 2.37s
678:	learn: 0.2679072	total: 5s	remaining: 2.36s
679:	learn: 0.2678335	total: 5.01s	remaining: 2.36s
680:	learn: 0.2677545	total: 5.02s	remaining: 2.35s
681:	learn: 0.2676963	total: 5.03s	remaining: 2.34s
682:	learn: 0.2675569	total: 5.03s	remaining: 2.33s
683:	learn: 0.2674630	total: 5.04s	remaining: 2.33s
684:	learn: 0.2673693	total: 5.05s	remaining: 2.32s
685:	learn: 0.2672793	total: 5.05s	remaining: 2.31s
686:	learn: 0.2672045	total: 5.06s	remaining: 2.31s
687:	learn: 0.2671063	total: 5.07s	remaining: 2.3s
688:	learn: 0.2669841	total: 5.08s	remaining: 2.29s
689:	learn: 0.2669017	total: 5.09s	remaining: 2.29s
690:	learn: 0.2668106	total: 5.1s	remaining: 2.28s
691:	learn: 0.2667315	total: 5.11s	remaining: 2.27s
692:	learn: 0.2666017	total: 5.12s	remaining: 2.27s
693:	learn: 0.2664844	total: 5.12s	remaining: 2.26s
694:	learn: 0.2664296	total: 5.13s	remaining: 2.25s
695:	learn: 0.2663429	total: 5.14s	remaining: 2.24s
696:	learn: 0.266

837:	learn: 0.2545559	total: 6.35s	remaining: 1.23s
838:	learn: 0.2544981	total: 6.36s	remaining: 1.22s
839:	learn: 0.2544255	total: 6.37s	remaining: 1.21s
840:	learn: 0.2543789	total: 6.37s	remaining: 1.2s
841:	learn: 0.2543064	total: 6.38s	remaining: 1.2s
842:	learn: 0.2542188	total: 6.38s	remaining: 1.19s
843:	learn: 0.2541530	total: 6.39s	remaining: 1.18s
844:	learn: 0.2541000	total: 6.4s	remaining: 1.17s
845:	learn: 0.2540375	total: 6.4s	remaining: 1.17s
846:	learn: 0.2539649	total: 6.41s	remaining: 1.16s
847:	learn: 0.2538330	total: 6.42s	remaining: 1.15s
848:	learn: 0.2537589	total: 6.42s	remaining: 1.14s
849:	learn: 0.2536962	total: 6.43s	remaining: 1.13s
850:	learn: 0.2536197	total: 6.44s	remaining: 1.13s
851:	learn: 0.2535858	total: 6.44s	remaining: 1.12s
852:	learn: 0.2535183	total: 6.45s	remaining: 1.11s
853:	learn: 0.2534394	total: 6.45s	remaining: 1.1s
854:	learn: 0.2533638	total: 6.46s	remaining: 1.09s
855:	learn: 0.2532741	total: 6.47s	remaining: 1.09s
856:	learn: 0.253

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.74      0.48      0.58       393

    accuracy                           0.86      2000
   macro avg       0.81      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000



In [12]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1362317415],
    'Geography': [5582848579876042824],
    'Gender': [5582604471130210632],
    'Age': [1364738365],
    'Tenure': [1383546173],
    'Balance': [1381055805],
    'NumOfProducts': [1380007229],
    'HasCrCard': [1380007229],
    'IsActiveMember': [1380007229],
    'EstimatedSalary': [5929072950456707169]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)


[1]


In [13]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1869898352],
    'Geography': [3473737896946584693],
    'Gender': [3473525673970054005],
    'Age': [1882809149],
    'Tenure': [1867595069],
    'Balance': [8092540212476270389],
    'NumOfProducts': [1883323709],
    'HasCrCard': [1883323709],
    'IsActiveMember': [1884372285],
    'EstimatedSalary': [8103214267097302333]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)


[0]


In [14]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1366183015],
    'Geography': [5582848579876042824],
    'Gender': [5582604471130210632],
    'Age': [1382961469],
    'Tenure': [1365720381],
    'Balance': [1381055805],
    'NumOfProducts': [1380007229],
    'HasCrCard': [1381055805],
    'IsActiveMember': [1380007229],
    'EstimatedSalary': [5861510164215982909]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)



[1]


In [15]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1866018666],
    'Geography': [8678483994702985277],
    'Gender': [3473525673970054005],
    'Age': [1868837949],
    'Tenure': [1883323709],
    'Balance': [8242847815683889469],
    'NumOfProducts': [1883323709],
    'HasCrCard': [1884372285],
    'IsActiveMember': [1883323709],
    'EstimatedSalary': [8092466536624052021]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)


[0]


In [16]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1366186098],
    'Geography': ["23957762034306829357020560701"],
    'Gender': [5711495825523686717],
    'Age': [1364738365],
    'Tenure': [1382497597],
    'Balance': [5867739997147851581],
    'NumOfProducts': [1383546173],
    'HasCrCard': [1380007229],
    'IsActiveMember': [1381055805],
    'EstimatedSalary': [6073179312342651197]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)


[1]


In [17]:
import pickle

# Assuming 'model' is your trained XGBClassifier
# Save the model to a file
with open('Cat_model.pickle', 'wb') as file:
    pickle.dump(cat_model, file)


In [18]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1866020713],
    'Geography': [3473737896946584693],
    'Gender': [3473525673970054005],
    'Age': [1868836925],
    'Tenure': [1886862653],
    'Balance': [1884372285],
    'NumOfProducts': [1883323709],
    'HasCrCard': [1883323709],
    'IsActiveMember': [1883323709],
    'EstimatedSalary': [8092457744812038010]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)


[0]


In [19]:
import pandas as pd

# Replace the values in the new DataFrame
new_data = pd.DataFrame({
    'CreditScore': [1867394928],
    'Geography': [3473737896946584693],
    'Gender': [3473525673970054005],
    'Age': [1868836925],
    'Tenure': [1917926717],
    'Balance': [8092531377746699575],
    'NumOfProducts': [1885814077],
    'HasCrCard': [1883323709],
    'IsActiveMember': [1884372285],
    'EstimatedSalary': [8092466532345858870]
})

# Make predictions on the new data
predictions = cat_model.predict(new_data)

# Display the predictions
print(predictions)


[1]
